In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
from os.path import isfile, join

import tensorflow.keras as keras
from tensorflow.keras import applications
from tensorflow.keras.layers import Flatten, LeakyReLU, Dense
from tensorflow.keras.models import *
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import random
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from datetime import date

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array


# training data
image_size = 224
#labels = pd.read_csv("data/fgvc7/train.csv")
image_dir = "data/leaf"

In [37]:
# import image data and combine labels
def load_unlabeled_data(image_dir):
    X = []
    for root, folder, files in os.walk(image_dir):
        #print(files)
        for f in files:
            #print(f)
            if f.endswith(".JPG"):
                #print(root, folder, f)
                img = load_img(f'{root}/{f}', target_size=(image_size,image_size,3))
                img_array = img_to_array(img, dtype='uint8')
                X.append(img_array)
    return np.array(X, dtype=np.uint16)

In [38]:
# import training data and combine labels
data = load_unlabeled_data(image_dir) 

KeyboardInterrupt: 

In [ ]:
import imgaug.augmenters as iaa
seq = iaa.Sequential([
    iaa.SomeOf((0,2),[
        iaa.Identity(),
        iaa.AverageBlur(k=((3, 5), (5, 7))),
        iaa.Rotate((-45,45)),
        iaa.Affine(scale=(0.5, 0.95)),    
        iaa.Multiply((0.50, 1.1))
        #,iaa.BlendAlphaRegularGrid(nb_rows=(4, 6), nb_cols=(1, 4),
        #                        foreground=iaa.Multiply(0.0))
        #,iaa.Cartoon()
        ,iaa.Cutout(nb_iterations=(1, 3), size=0.2, squared=False, cval=0)
        ,iaa.Affine(shear=(-48, 48))
        ,iaa.Affine(translate_px={"x": (-42, 42), "y": (-36, 36)})
        ,iaa.KeepSizeByResize(iaa.Resize({"height": (0.70, 0.90), "width": (0.70, 0.90)}))
        ,iaa.CropAndPad(percent=(-0.2, 0.2))
        #,iaa.PiecewiseAffine(scale=(0.01, 0.05))
        ,iaa.PerspectiveTransform(scale=(0.01, 0.1))
        #,iaa.WithPolarWarping(iaa.CropAndPad(percent=(-0.1, 0.1)))
        #,iaa.ElasticTransformation(alpha=(0, 3.0), sigma=0.5)
       ])
    #,iaa.SaveDebugImageEveryNBatches(folder_path, 100)    
], random_order=True)

In [ ]:
def generator(features, batch_size):
    while True:
        # Fill arrays of batch size with augmented data taken randomly from full passed arrays
        indexes = random.sample(range(len(features)), batch_size)
      
        # Transform X1 and X2
        x_aug_1 = seq(images =features[indexes])
        x_aug_2 = seq(images =features[indexes])
        yield np.array(x_aug_1), np.array(x_aug_2), np.array(labels[indexes])

In [ ]:
images = generator(x1,x2, 9)
image = next(images)

plt.figure(figsize=(10, 10))
for i in range(9):
    ax1 = plt.subplot(1, 9, i+1)
    plt.imshow(image[0][i])
    ax2 = plt.subplot(2, 9, i+1)
    plt.imshow(image[1][i])
plt.axis("off")